In [5]:
from astropy.cosmology import LambdaCDM
import numpy as np

In [6]:
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7, Tcmb0=2.725)

In [9]:
z = np.arange(0, 1.5, 0.01)

In [12]:
phi_star = 3.6 * cosmo.efunc(z)**2

In [13]:
alpha = -1.05 * (1 + z)**(-2/3)

In [14]:
fr = 0.8*(1 + z)**(-1/2)